# Prepare Input Data
We are going to train an MPNN to predict the B3LYP-level atomization energy.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from moldesign.score.mpnn.data import convert_nx_to_dict, make_tfrecord
from moldesign.utils.conversions import convert_string_to_nx
from moldesign.store.mongo import MoleculePropertyDB
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import json
import os

RDKit WARNING: [09:53:26] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
mongo = MoleculePropertyDB.from_connection_info(port=27846)
print(f'Found {len(mongo.get_molecules())} molecules')

Found 6523 molecules


In [4]:
!hostname

beboplogin4


## Get the Data
The best copy is in our MongoDB isntance

Get a mapping of smiles to "normal basis"

In [5]:
mongo.update_derived_properties()

In [8]:
data = mongo.get_training_set(['key', 'identifier.smiles', 'data.small_basis.neutral.xyz', 'data.small_basis.reduced.xyz', 'reduction_potential.smb-vacuum-vertical',
                              'reduction_potential.smb-vacuum-no-zpe'], ['reduction_potential.dfb-acn-smb-geom'])
data = pd.DataFrame(data)
print(f'Loaded {len(data)} molecules')

Loaded 3188 molecules


Convert the SMILES to a networkx object

In [9]:
%%time
data['nx'] = data['identifier.smiles'].apply(convert_string_to_nx)

CPU times: user 1.47 s, sys: 15.5 ms, total: 1.48 s
Wall time: 1.61 s


## Save the Data as TF Records
We'll make both a training, validation and test set

In [10]:
train_set, test_set = train_test_split(data, test_size=0.1, random_state=1, shuffle=True)
print(f'Set aside {len(test_set)} training entries')

Set aside 319 training entries


In [11]:
train_set, val_set = train_test_split(train_set, test_size=0.1, random_state=1, shuffle=True)
print(f'Split off {len(train_set)} training and {len(val_set)} validation entries')

Split off 2582 training and 287 validation entries


Get all of the types observed in the dataset

Save the data in TF format

In [12]:
os.makedirs('datasets', exist_ok=True)

In [13]:
for name, dataset in zip(['train', 'valid', 'test'], [train_set, val_set, test_set]):
    dataset = dataset.sample(frac=1.)  # Shuffle contents
    dataset.drop(columns=['nx']).to_csv(f'datasets/{name}.csv', index=False)
    with tf.io.TFRecordWriter(f'datasets/{name}_data.proto') as writer:
        for _, entry in tqdm(dataset.iterrows(), desc=name):
            record = convert_nx_to_dict(entry['nx'])
            record['output'] = entry['reduction_potential.dfb-acn-smb-geom']
            writer.write(make_tfrecord(record))

train: 215it [00:00, 2142.19it/s]

ERROR! Session/line number was not unique in database. History logging moved to new session 4458


train: 2582it [00:01, 2184.70it/s]
valid: 287it [00:00, 2020.81it/s]
test: 319it [00:00, 2059.17it/s]
